In [3]:
file_path1 = "C:\\Users\\nidhi\\OneDrive\\Documents\\PracticeCode\\File1.xlsx"
file_Path2 = "C:\\Users\\nidhi\\Documents\\PracticeCode\\File2.xlsx"

parquet_path1 = "C:\\Users\\nidhi\\Documents\\PracticeCode\\File1.Parquet"
parquet_path2 = "C:\\Users\\nidhi\\Documents\\PracticeCode\\File2.Parquet"

In [4]:
import pandas as pd

file1_df = pd.read_excel(file_path1)
display(file1_df)

,RecType,Pk.Fund,BonusSeries,ck.IssYear,Value1
0,(t),NaN,NaN,NaN,1.0
1,data,ALBA,BL_KJ,ALL,0.9
2,data,ALBA,BL_KJ,ALL,0.9
3,data,ALBA,BL_KJ,ALL,0.9
4,data,ALBA,BL_KJ,ALL,0.9


In [37]:
import pandas as pd

file1_df = pd.read_excel(file_path1)
file2_df = pd.read_excel(file_path1)
display(file1_df.iloc[:1])
display(file1_df)

,RecType,Pk.Fund,BonusSeries,ck.IssYear,Value1
0,(t),NaN,NaN,NaN,1.0


,RecType,Pk.Fund,BonusSeries,ck.IssYear,Value1
0,(t),NaN,NaN,NaN,1.0
1,data,ALBA,BL_KJ,ALL,0.9
2,data,ALBA,BL_KJ,ALL,0.9
3,data,ALBA,BL_KJ,ALL,0.9
4,data,ALBA,BL_KJ,ALL,0.9


In [5]:
special_col = 'RECTYPE'
file1_df.columns = [col.upper() for col in file1_df.columns]
print(file1_df.columns)

Index(['RECTYPE', 'PK.FUND', 'BONUSSERIES', 'CK.ISSYEAR', 'VALUE1'], dtype='object')


In [6]:
display(file1_df)

,RECTYPE,PK.FUND,BONUSSERIES,CK.ISSYEAR,VALUE1
0,(t),NaN,NaN,NaN,1.0
1,data,ALBA,BL_KJ,ALL,0.9
2,data,ALBA,BL_KJ,ALL,0.9
3,data,ALBA,BL_KJ,ALL,0.9
4,data,ALBA,BL_KJ,ALL,0.9


In [39]:
def get_vb_cols_list(df, special_list:list):
    return[ c for c in df.columns if (c not in special_list) and not ((c[:5] == "VALUE") and c[5:].isdigit())
          ]

In [40]:
vb_cols_list = get_vb_cols_list(file1_df, [special_col])
print(vb_cols_list)

['PK.FUND', 'BONUSSERIES', 'CK.ISSYEAR']


In [41]:
possible_indices = file1_df.iloc[:2][special_col]
print(possible_indices)
if possible_indices.iloc[1] == 'data':
    index_len = 1
else:
    index_len = 2
print(index_len)

0     (t)
1    data
Name: RECTYPE, dtype: object
1


In [42]:
def handle_ait_index_data_types(file1_df, index_len, special_col):
    indices = {}
    cols = file1_df.iloc[:index_len][[c for c in file1_df.columns if c not in ['RECTYPE'] + get_vb_cols_list(file1_df, [special_col])
    ]]
    print(cols)
    for i in range(index_len):
        try:
            vals =cols.iloc[i].values.astype("int32")
        except ValueError:
            vals =cols.iloc[i].values
        indices[file1_df[special_col].iloc[i]] = vals
    return indices
    

In [43]:
indices = handle_ait_index_data_types(file1_df, index_len, special_col)
print(indices)


   VALUE1
0     1.0
{'(t)': array([1])}


In [44]:
import numpy as np
vals = file1_df.iloc[index_len:][[c for c in file1_df.columns if c not in [special_col] +vb_cols_list]].values
vals = vals.astype(np.float64)
print(vals)

[[0.9]
 [0.9]
 [0.9]
 [0.9]]


In [45]:
vb_cols = file1_df.iloc[index_len:][vb_cols_list]
vb_cols = vb_cols.assign(**{col: vb_cols[col].str.upper() for col in vb_cols.columns})
print(vb_cols)

  PK.FUND BONUSSERIES CK.ISSYEAR
1    ALBA       BL_KJ        ALL
2    ALBA       BL_KJ        ALL
3    ALBA       BL_KJ        ALL
4    ALBA       BL_KJ        ALL


In [46]:
vb_cols = vb_cols.assign(**{col: vb_cols[col].astype('category') for col in vb_cols.columns})
print(vb_cols)

  PK.FUND BONUSSERIES CK.ISSYEAR
1    ALBA       BL_KJ        ALL
2    ALBA       BL_KJ        ALL
3    ALBA       BL_KJ        ALL
4    ALBA       BL_KJ        ALL


In [47]:
index_df = pd.DataFrame.from_dict({k:v for k,v in indices.items()})
display(index_df)
index = pd.MultiIndex.from_frame(index_df)
display(index)


,(t)
0,1


MultiIndex([(1,)],
           names=['(t)'])

In [56]:
core_df = pd.DataFrame(index=pd.MultiIndex.from_frame(vb_cols),data=vals, columns=index)
display(core_df)
tota_rows = core_df.shape[0]
tota_cols = core_df.shape[1]
print(tota_rows)
print(tota_cols)

(t)                               1
PK.FUND BONUSSERIES CK.ISSYEAR     
ALBA    BL_KJ       ALL         0.9
                    ALL         0.9
                    ALL         0.9
                    ALL         0.9

4
1


In [53]:
pd.melt(core_df, ignore_index = False)

(t)  value
PK.FUND BONUSSERIES CK.ISSYEAR            
ALBA    BL_KJ       ALL           1    0.9
                    ALL           1    0.9
                    ALL           1    0.9
                    ALL           1    0.9

In [52]:
core_df.unpivot().to_parquet(parquet_path1)

AttributeError: 'DataFrame' object has no attribute 'unpivot'

In [7]:
file1_df.to_parquet(parquet_path1)
file2_df.to_parquet(parquet_path2)

In [8]:
file1_parquet_df = pd.read_parquet(parquet_path1)
file2_parquet_df = pd.read_parquet(parquet_path1)

In [9]:
display(file1_parquet_df)

,RecType,Pk.Fund,BonusSeries,ck.IssYear,Value1
0,(t),None,None,None,1.0
1,data,ALBA,BL_KJ,ALL,0.9
2,data,ALBA,BL_KJ,ALL,0.9
3,data,ALBA,BL_KJ,ALL,0.9
4,data,ALBA,BL_KJ,ALL,0.9
